In [26]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [52]:
import os
import re

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [28]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [29]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [30]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [31]:
words = ["pizza", "computer", "queen", "forest"]

In [32]:
for word in words:
    print(f"\nMost similar words to '{word}':")
    similar = glove_wiki_vectors.most_similar(word, topn=5)
    for sim_word, score in similar:
        print(f"  {sim_word} (similarity: {score:.4f})")


Most similar words to 'pizza':
  sandwich (similarity: 0.7488)
  sandwiches (similarity: 0.7222)
  bread (similarity: 0.7007)
  pizzas (similarity: 0.7005)
  burger (similarity: 0.6978)

Most similar words to 'computer':
  computers (similarity: 0.8752)
  software (similarity: 0.8373)
  technology (similarity: 0.7642)
  pc (similarity: 0.7366)
  hardware (similarity: 0.7290)

Most similar words to 'queen':
  princess (similarity: 0.7947)
  king (similarity: 0.7508)
  elizabeth (similarity: 0.7356)
  royal (similarity: 0.7065)
  lady (similarity: 0.7045)

Most similar words to 'forest':
  forests (similarity: 0.8435)
  woodland (similarity: 0.7261)
  rainforest (similarity: 0.6805)
  wildlife (similarity: 0.6762)
  habitat (similarity: 0.6734)


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [33]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [34]:
for w1, w2 in word_pairs:
    try:
        similarity = glove_wiki_vectors.similarity(w1, w2)
        print(f"Cosine similarity between '{w1}' and '{w2}': {similarity:.4f}")
    except KeyError as e:
        print(f"One or both the words ('{word1}', '{word2}') are not a part of the vocabulary.")

Cosine similarity between 'coast' and 'shore': 0.7000
Cosine similarity between 'clothes' and 'closet': 0.5463
Cosine similarity between 'old' and 'new': 0.6432
Cosine similarity between 'smart' and 'intelligent': 0.7553
Cosine similarity between 'dog' and 'cat': 0.8798
Cosine similarity between 'tree' and 'lawyer': 0.0767


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [35]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [36]:
for word in test_words:
    in_vocab = word in glove_wiki_vectors
    print(f"{word}: {'In vocabulary' if in_vocab else 'Not in vocabulary'}")


covididiot: Not in vocabulary
fomo: Not in vocabulary
frenemies: In vocabulary
anthropause: Not in vocabulary
photobomb: Not in vocabulary
selfie: Not in vocabulary
pxg: Not in vocabulary
pacg: Not in vocabulary
cct: In vocabulary
escc: In vocabulary


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [37]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [38]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [39]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283238

In [40]:
glove_wiki_vectors.similarity("white", "success")

0.351824

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [41]:
print("Bias in beauty and gender:")
display(analogy("man", "strong", "woman", model=glove_wiki_vectors))

Bias in beauty and gender:
man : strong :: woman : ?


,Analogy word,Score
0,stronger,0.726966
1,weak,0.657428
2,robust,0.644944
3,strongest,0.634013
4,despite,0.631509
5,support,0.627726
6,growing,0.625404
7,concern,0.605287
8,particularly,0.603681
9,reflected,0.600039


In [42]:
print("Bias in gender and family roles:")
display(analogy("man", "boss", "woman", model=glove_wiki_vectors))

Bias in gender and family roles:
man : boss :: woman : ?


,Analogy word,Score
0,bosses,0.649429
1,girlfriend,0.639793
2,boyfriend,0.623138
3,colleague,0.604826
4,lover,0.593473
5,husband,0.590123
6,ex,0.575455
7,friend,0.565926
8,wife,0.555912
9,housekeeper,0.527948


In [43]:
print("Bias in religion and violence:")
display(analogy("christian", "peace", "muslim", model=glove_wiki_vectors))

Bias in religion and violence:
christian : peace :: muslim : ?


,Analogy word,Score
0,truce,0.679213
1,kashmir,0.660678
2,ceasefire,0.656898
3,conflict,0.656596
4,cease,0.645088
5,palestinian,0.644061
6,sudan,0.633034
7,darfur,0.628929
8,palestinians,0.628015
9,sudanese,0.618921


In [44]:
print("Bias in race and crime association:")
display(analogy("white", "lawful", "black", model=glove_wiki_vectors))

Bias in race and crime association:
white : lawful :: black : ?


,Analogy word,Score
0,unlawful,0.703117
1,legitimate,0.607818
2,unjust,0.545922
3,arbitrary,0.542710
4,unreasonable,0.538038
5,capricious,0.521254
6,legally,0.520878
7,immoral,0.518508
8,coercive,0.503806
9,cruel,0.502898


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

### **Key Takeaways on Bias in Word Embeddings**

#### Observed Biases:
1. **Gender Bias**  
   Words like *"woman"* are frequently associated with caregiving roles such as *"nurse"* or *"mother"*, while *"man"* tends to be linked with traits like *"strong"* or professions like *"doctor"*, reflecting traditional gender stereotypes.

2. **Cultural Bias**  
   Female-associated terms, such as *"queen"*, are more often connected to words like *"beautiful"* or *"glamorous"*, emphasizing appearance over power or status, in contrast to male terms like *"king"*.

3. **Racial Bias**  
   Words like *"white"* show a stronger semantic association with terms like *"success"*, while words like *"black"* tend to have weaker or more negative associations, indicating implicit racial bias.

---

#### Why This Matters:
Pre-trained word embeddings reflect patterns in the data they are trained on, including societal stereotypes. These biases can carry over into machine learning models and affect their predictions and decisions.

---

#### Potential Harms in Real-World Applications:
- **Hiring Algorithms**: May reinforce existing gender or racial disparities in candidate evaluation.
- **Healthcare Systems**: Risk of unequal treatment prioritization based on biased language data.
- **Content Moderation**: Biased embeddings may fail to accurately detect or classify offensive content across different groups.
- **Generative AI Models**: May produce outputs that unintentionally reinforce stereotypes and biased narratives.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [45]:
from sklearn.datasets import fetch_20newsgroups

In [46]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [47]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [48]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [53]:
def preprocess_texts_with_pipe(texts):
    processed_texts = []
    for doc in nlp.pipe(texts, batch_size=50):  # Process in batches
        tokens = [
            token.lemma_.lower()
            for token in doc
            if not token.is_stop
            and not token.is_punct
            and not token.like_url
            and not token.like_email
            and not token.like_num
            and token.is_alpha
        ]
        processed_texts.append(" ".join(tokens))
    return processed_texts

def clean_raw_text(text):
    text = re.sub(r"\S+@\S+", "EMAIL", text)
    text = re.sub(r"http\S+|www\S+", "URL", text)
    text = re.sub(r"\d+", "NUM", text)
    text = re.sub(r"[\r\n\t\/\\]", " ", text)
    return text

df["text_cleaned"] = df["text"].apply(clean_raw_text)

df["text_pp"] = preprocess_texts_with_pipe(df["text_cleaned"])

df.to_csv("preprocessed_text.csv", index=False)

In [54]:
df.iloc[2:6]

,text,target,target_name,text_cleaned,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,"Actuallay I don't, but on the other hand I do...",actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,"The following problem is really bugging me, an...",follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,This is the latest from UPI Foreign...,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,"Hi, I'd like to subscribe to Leadership Mag...",hi like subscribe leadership magazine wonder d...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

### **Preprocessing Steps**

- **Initial Text Cleaning (Regex)**:  
  Replaced noisy patterns such as email addresses, URLs, numeric values, and escape characters with generic placeholders.  
  *Prevents rare, non-informative tokens from distorting topic distributions.*

- **Tokenization**:  
  Used `spaCy`'s `nlp.pipe()` to efficiently split the text into individual tokens.  
  *Ensures consistent and efficient document parsing.*

- **Lemmatization**:  
  Converted each token to its lemma (base form) and lowercased it.  
  *Groups word variants (e.g., “running” and “run”) to improve topic clustering.*

- **Stopword Removal**:  
  Removed common English stopwords (e.g., "and", "is", "but").  
  *Eliminates high-frequency but low-value words.*

- **Punctuation & Special Token Filtering**:  
  Removed punctuation, numeric tokens, and other non-alphabetic strings.  
  *Reduces clutter and keeps the token set semantically meaningful.*

- **Batch Processing with `nlp.pipe()`**:  
  Used spaCy’s optimized pipeline to process texts in batches.  
  *Improves runtime performance on large corpora.*

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

### **Justification for Number of Topics**

- **Exploration**:  
  We trained multiple LDA models with different topic counts (`n_components = 5, 8, 10, 15`) using `random_state=42`.

- **Clarity & Coherence**:  
  - Models with **fewer topics** (e.g., 5) grouped unrelated themes together, resulting in overly broad topics.  
  - Models with **too many topics** (e.g., 10 or 15) introduced overlap and fragmentation, making interpretation more difficult.

- **Final Choice – 8 Topics**:  
  - **8 topics** aligned well with the known number of categories in our dataset (e.g., religion, sports, politics, tech).  
  - The topics generated were coherent, distinguishable, and easy to label based on top keywords.  
  - This number balanced interpretability with sufficient thematic granularity.

In [55]:
from sklearn.decomposition import LatentDirichletAllocation

In [56]:
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = vectorizer.fit_transform(df["text_pp"])

topic_numbers = [5, 8, 10, 15]
lda_models = {}

for n in topic_numbers:
    lda = LatentDirichletAllocation(
        n_components=n,
        random_state=42,
        learning_method="batch",
        max_iter=10
    )
    lda.fit(dtm)
    lda_models[n] = lda

selected_n_topics = 8
lda_model = lda_models[selected_n_topics]

def display_topics(model, feature_names, num_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]
        topics[f"Topic {topic_idx + 1}"] = top_words
    return topics

num_top_words = 10
feature_names = vectorizer.get_feature_names_out()

topics = display_topics(lda_model, feature_names, num_top_words)
topics

{'Topic 1': ['team',
  'game',
  'year',
  'good',
  'run',
  'player',
  'win',
  'like',
  'play',
  'think'],
 'Topic 2': ['num',
  'period',
  'email',
  'la',
  'play',
  'team',
  'pts',
  'new',
  'pp',
  'power'],
 'Topic 3': ['num',
  'people',
  'armenian',
  'turkish',
  'jews',
  'armenians',
  'claim',
  'turkey',
  'human',
  'state'],
 'Topic 4': ['say',
  'num',
  'know',
  'people',
  'come',
  'tell',
  'think',
  'like',
  'time',
  'want'],
 'Topic 5': ['num',
  'email',
  'file',
  'program',
  'use',
  'image',
  'window',
  'available',
  'include',
  'server'],
 'Topic 6': ['god',
  'jesus',
  'believe',
  'people',
  'think',
  'know',
  'say',
  'num',
  'thing',
  'bible'],
 'Topic 7': ['game',
  'year',
  'think',
  'good',
  'play',
  'team',
  'time',
  'season',
  'player',
  'like'],
 'Topic 8': ['gun',
  'israel',
  'people',
  'right',
  'state',
  'law',
  'weapon',
  'num',
  'think',
  'firearm']}

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [57]:
topics_df = pd.DataFrame.from_dict(
    topics,
    orient='index',
    columns=[f"Word {i+1}" for i in range(num_top_words)]
)

In [58]:
print(topics_df)

        Word 1  Word 2    Word 3   Word 4 Word 5     Word 6  Word 7  \
Topic 1   team    game      year     good    run     player     win   
Topic 2    num  period     email       la   play       team     pts   
Topic 3    num  people  armenian  turkish   jews  armenians   claim   
Topic 4    say     num      know   people   come       tell   think   
Topic 5    num   email      file  program    use      image  window   
Topic 6    god   jesus   believe   people  think       know     say   
Topic 7   game    year     think     good   play       team    time   
Topic 8    gun  israel    people    right  state        law  weapon   

            Word 8   Word 9  Word 10  
Topic 1       like     play    think  
Topic 2        new       pp    power  
Topic 3     turkey    human    state  
Topic 4       like     time     want  
Topic 5  available  include   server  
Topic 6        num    thing    bible  
Topic 7     season   player     like  
Topic 8        num    think  firearm  


| **Topic #** | **Top Words** | **Suggested Label** |
|------------|----------------|---------------------|
| **Topic 1** | team, game, year, good, run, player, win, like, play, think | Sports (General) |
| **Topic 2** | num, period, email, la, play, team, pts, new, pp, power | Emails / Sports Statistics |
| **Topic 3** | num, people, armenian, turkish, jews, armenians, claim, turkey, human, state | Ethnic and Geopolitical Conflict |
| **Topic 4** | say, num, people, come, tell, think, time, like, want | General Discussion / Opinion |
| **Topic 5** | num, email, file, program, use, image, window, available, include, server | Technology / File Systems |
| **Topic 6** | god, jesus, believe, people, think, know, say, num, thing, bible | Religion and Belief |
| **Topic 7** | game, year, think, good, play, team, time, season, player, like | Sports |
| **Topic 8** | gun, israel, people, right, state, law, weapon, num, think, firearm | Guns, Politics, and Law |

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [59]:
doc_topic_matrix = lda_model.transform(dtm)

df["Dominant_Topic"] = np.argmax(doc_topic_matrix, axis=1) + 1  # +1 to match "Topic 1" style indexing
df["Topic_Probability"] = np.max(doc_topic_matrix, axis=1)

first_five_docs = df[["text", "text_pp", "Dominant_Topic", "Topic_Probability"]].head()
print(first_five_docs)

                                                text  \
0  You know, I was reading 18 U.S.C. 922 and some...   
1  \n\n\nIt's not a bad question: I don't have an...   
2  \nActuallay I don't, but on the other hand I d...   
3  The following problem is really bugging me,\na...   
4  \n\n  This is the latest from UPI \n\n     For...   

                                             text_pp  Dominant_Topic  \
0  know read num num sence wonder help num num pr...               8   
1  bad question ref list algorithm think bit hard...               5   
2  actuallay hand support idea have newsgroup asp...               6   
3  follow problem bug appreciate help create wind...               5   
4  late upi foreign ministry spokesman ferhat ata...               3   

   Topic_Probability  
0           0.889240  
1           0.988151  
2           0.576054  
3           0.831263  
4           0.730968  


### **Document Topic Interpretation**

#### 1. **Dominant Topic Interpretation:**

- **Document 0**: Assigned to **Topic 8** with **high confidence (0.89)**. The text discusses U.S. law (e.g., 18 U.S.C. 922), which aligns well with Topic 8’s focus on **guns, law, and political rights**.
  
- **Document 1**: Assigned to **Topic 5** with **very high confidence (0.99)**. The keywords *algorithm*, *list*, and *think* suggest a strong alignment with **computing or programming-related content**.

- **Document 2**: Assigned to **Topic 6** with **moderate confidence (0.58)**. Although not highly confident, the text’s structure and terms like *support*, *idea*, and *newsgroup* could relate to **online forums or belief systems**, which fits with Topic 6’s broader theme around **opinions or social discourse**.

- **Document 3**: Assigned to **Topic 5** with **high confidence (0.83)**. It discusses a technical bug and solutions, which matches well with **software, user interface issues, or system troubleshooting**.

- **Document 4**: Assigned to **Topic 3** with **good confidence (0.73)**. The document refers to foreign affairs and UPI news, likely matching Topic 3’s focus on **geopolitical or international conflict themes**.

---

#### 2. **Strengths:**

- The model identifies **computing and law-related documents** with high clarity and confidence (e.g., Documents 0, 1, and 3).
- **Topic 3** successfully captures political/geopolitical content in Document 4, showing that domain-specific language is being clustered effectively.
- Confidence scores above **0.80** in 3 out of 5 cases suggest strong topic alignment.

---

#### 3. **Weaknesses:**

- **Document 2** has a **lower confidence score (~0.58)**, indicating weaker topic clarity — possibly due to **generic or conversational phrasing**.
- Some overlap between topics involving **technology, support, or community-related vocabulary** may reduce distinction (e.g., Document 2 vs. Document 3).
- Topics involving **broad social themes** may be harder to isolate without stricter preprocessing.

---

#### 4. **Recommendations:**

- Apply additional text cleaning (e.g., removing repeated “num”-like placeholders) to improve clarity in documents like Document 2.
- Explore **part-of-speech (POS) filtering** to retain more informative tokens (e.g., nouns and verbs only).
- Consider **splitting broad topics** or increasing the number of topics slightly if you notice consistent overlap between categories like community support vs. technical bugs.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

#### **1. How Content-Based Filtering Works:**

Content-based filtering recommends items to a user by analyzing the attributes of items they have previously interacted with or liked. It builds a **user profile** based on features of those items (e.g., genre, keywords, tags), then compares that profile to other items in the system and suggests ones with similar content.

For example, if a user watches several sci-fi movies, the system recommends other sci-fi movies based on shared features like “space,” “futuristic,” or “AI.”

---

#### **2. Negative Consequences of Recommender Systems:**

- **Filter Bubbles & Echo Chambers**:  
  Recommender systems often reinforce a user’s existing preferences, exposing them to only similar viewpoints or content. This can lead to **narrowed perspectives** and lack of diversity in information.

- **Bias & Inequality**:  
  Systems may unintentionally favor popular or well-represented items, making it harder for **niche or minority content** to get visibility. This can amplify societal biases and reduce fairness in recommendations.

---

#### **3. What is Transfer Learning in NLP?**

Transfer learning in NLP involves using a **pre-trained language model** (e.g., BERT, GPT) that has already learned general language patterns from a large corpus, and then **fine-tuning** it on a specific task like sentiment analysis or question answering. This approach leverages prior knowledge to achieve better performance with less task-specific data.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)